# MPIA Arxiv on Deck 2: reprocessing

This notebook reprocesses some papers with the current baseline.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper to reprocess

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname

In [5]:
from glob import glob
paper_refs = [k.split('/')[-1].replace('.md',  '') for k in glob('_build/html/**md')]
n_papers = len(paper_refs)
print(f"Found {n_papers:,d} from past processing.")

Found 192 from past processing.


In [6]:
new_papers = [get_paper_from_identifier(k) for k in tqdm(paper_refs)]

# select only papers with matching author names and highlight authors
candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

  0%|          | 0/192 [00:00<?, ?it/s]

Arxiv has 192 new papers today
          192 with possible author matches


In [7]:
# move papers that are not matching

import os
import shutil

storage = "_build/html"

try:
    os.makedirs(os.path.join(storage, "removed"))
except FileExistsError:
    pass

for paper in new_papers:
    if paper not in candidates:
        print(f"moving {paper['identifier']} to {storage}/removed")
        shutil.move(os.path.join(storage, paper['identifier'] + '.md'),
                    os.path.join(storage, "removed", paper['identifier'] + '.md'))
        shutil.move(os.path.join(storage, "tmp_" + paper['identifier']),
                    os.path.join(storage, "removed", "tmp_" + paper['identifier']))

# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [8]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        # highlight authors (FIXME: doc.highlight_authors)
        doc._authors = highlight_authors_in_list(
            [mpia.get_initials(k) for k in doc.authors], 
            normed_mpia_authors, verbose=True)
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/191 [00:00<?, ?it/s]

/Users/fouesneau/Work/arxiv_display/venv/lib/python3.11/site-packages/arxiv_on_deck_2/latex.py:359: LatexWarning: Latex injecting: 'ngc1365_center_astro-ph.bbl' from 'tmp_2212.09168/ngc1365_center_astro-ph.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


Found 129 bibliographic references in tmp_2212.09168/ngc1365_center_astro-ph.bbl.


/Users/fouesneau/Work/arxiv_display/venv/lib/python3.11/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Leroy} {et~al.}(subm.{\natexlab{a}})}]{LEROY2_PHANGSJWST}{Leroy}, A., {et~al.} subm.{\natexlab{a}}, \apjl
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/Users/fouesneau/Work/arxiv_display/venv/lib/python3.11/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Leroy} {et~al.}(subm.{\natexlab{b}})}]{LEROY1_PHANGSJWST}---. subm.{\natexlab{b}}, \apjl
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/Users/fouesneau/Work/arxiv_display/venv/lib/python3.11/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item =

Found 89 bibliographic references in tmp_2301.04442/arxiv.bbl.
Found 70 bibliographic references in tmp_2209.02725/main.bbl.


...

extracting tarball to tmp_2302.10008... done.


/Users/fouesneau/Work/arxiv_display/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/fouesneau/Work/arxiv_display/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2302.10008/toi3235_aas.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/fouesneau/Work/arxiv_display/venv/lib/python3.11/site-packages/arxiv_on_deck_2/latex.py:359: LatexWarning: Latex injecting: 'rvtable.tex' from 'tmp_2302.10008/rvtable.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/Users/fouesneau/Work/arxiv_display/venv/lib/python3.11/site-packages/arxiv_on_deck_2/latex.py:359: LatexWarning: Latex injecting: 'phfu_tab_short.tex' from 'tmp_2302.10008/phfu_tab_short.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/Users/fouesneau/Work/arxiv_display/venv/lib/p

Found 98 bibliographic references in tmp_2302.10008/toi3235_aas.bbl.
Retrieving document from  https://arxiv.org/e-print/2212.02235
extracting tarball to tmp_2212.02235... done.
Found 109 bibliographic references in tmp_2212.02235/aanda.bbl.
Found 50 bibliographic references in tmp_2208.08872/CNN_Giraffe.bbl.
Retrieving document from  https://arxiv.org/e-print/2212.10512
extracting tarball to tmp_2212.10512... done.
Found 63 bibliographic references in tmp_2212.10512/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2212.04026


/Users/fouesneau/Work/arxiv_display/venv/lib/python3.11/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Chastenet} {et~al.}(subm.)}]{CHASTENET1_PHANGSJWST}{Chastenet}, J., {et~al.} subm., \apjl
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/Users/fouesneau/Work/arxiv_display/venv/lib/python3.11/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Groves} {et~al.}(subm.)}]{GROVES_HIICAT}{Groves}, B., {et~al.} subm., \mnras
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/Users/fouesneau/Work/arxiv_display/venv/lib/python3.11/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Leroy} {et~al.}(

extracting tarball to tmp_2212.04026... done.
list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [9]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/reprocessing-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2212.09168-b31b1b.svg)](https://arxiv.org/abs/2212.09168) | **PHANGS-JWST First Results: Rapid Evolution of Star Formation in the Central Molecular Gas Ring of NGC1365**  |
|| <mark>Eva Schinnerer</mark>, et al. -- incl., <mark>Stephen Hannon</mark>, <mark>Justus Neumann</mark>, <mark>Sophia K. Stuber</mark> |
|*Appeared on*| *2022-12-18*|
|*Comments*| *24 pages, 8 figures, accepted for publications as part of PHANGS-JWST ApJL Focus Issue*|
|**Abstract**| Large-scale bars can fuel galaxy centers with molecular gas, often leading tothe development of dense ring-like structures where intense star formationoccurs, forming a very different environment compared to galactic disks. Wepair ~0.3" (30pc) resolution new JWST/MIRI imaging with archival ALMA CO(2-1)mapping of the central ~5kpc of the nearby barred spiral galaxy NGC1365, toinvestigate the physical mechanisms responsible for this extreme starformation. The molecular gas morphology is resolved into two well-known brightbar lanes that surround a smooth dynamically cold gas disk (R_gal ~ 475pc)reminiscent of non-star-forming disks in early type galaxies and likely fed bygas inflow triggered by stellar feedback in the lanes. The lanes host a largenumber of JWST-identified massive young star clusters. We find some evidencefor temporal star formation evolution along the ring. The complex kinematics inthe gas lanes reveal strong streaming motions and may be consistent withconvergence of gas streamlines expected there. Indeed, the extreme line-widthsare found to be the result of inter-`cloud' motion between gas peaks; ScousePydecomposition reveals multiple components with line widths of <sigma_CO,scouse>~ 19km/s and surface densities of <Sigma_H2,scouse> ~ 800M_sun/pc^2, similar tothe properties observed throughout the rest of the central molecular gasstructure. Tailored hydro-dynamical simulations exhibit many of the observedproperties and imply that the observed structures are transient and highlytime-variable. From our study of NGC1365, we conclude that it is predominantlythe high gas inflow triggered by the bar that is setting the star formation inits CMZ.|


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2301.04442-b31b1b.svg)](https://arxiv.org/abs/2301.04442) | **The CARMENES search for exoplanets around M dwarfs. A long-period planet around GJ 1151 measured with CARMENES and HARPS-N data**  |
|| J. Blanco-Pozo, et al. -- incl., <mark>M. Pérez-Torres</mark>, <mark>E. L. Brown</mark>, <mark>M. Kürster</mark> |
|*Appeared on*| *2023-01-11*|
|*Comments*| *18 pages, 11 figures. Accepted version, A&A (2023)*|
|**Abstract**| Detecting a planetary companion in a short-period orbit through radioemission from the interaction with its host star is a new prospect in exoplanetscience. Recently, a tantalising signal was found close to the low-mass stellarsystem GJ 1151 using LOFAR observations. We studied spectroscopic time-seriesdata of GJ 1151 in order to search for planetary companions, investigatepossible signatures of stellar magnetic activity, and to find possibleexplanations for the radio signal. We used the combined radial velocitiesmeasured from spectra acquired with the CARMENES, HARPS-N, and HPF instruments,extracted activity indices from those spectra in order to mitigate the impactof stellar magnetic activity on the data, and performed a detailed analysis ofGaia astrometry and all available photometric time series coming from theMEarth and ASAS-SN surveys. We found a M$>$10.6 M$_{\oplus}$ companion to GJ1151 in a 390d orbit at a separation of 0.57 au. Evidence for a secondmodulation is also present; this could be due to long-term magnetic variabilityor a second (substellar) companion. The star shows episodes of elevatedmagnetic activity, one of which could be linked to the observed LOFAR radioemission. We show that it is highly unlikely that the detected GJ 1151 b, orany additional outer companion is the source of the detected signal. We cannotfirmly rule out the suggested explanation of an undetected short-period planetthat could be related to the radio emission, as we establish an upper limit of1.2 M$_{\oplus}$ for the minimum mass.|


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2209.02725-b31b1b.svg)](https://arxiv.org/abs/2209.02725) | **A Generative Model for Quasar Spectra**  |
|| Anna-Christina Eilers, et al. -- incl., <mark>Frederick B. Davies</mark> |
|*Appeared on*| *2022-09-06*|
|*Comments*| *accepted for publication in ApJ*|
|**Abstract**| We build a multi-output generative model for quasar spectra and theproperties of their black hole engines, based on a Gaussian processlatent-variable model. This model treats every quasar as a vector of latentproperties such that the spectrum and all physical properties of the quasar areassociated with non-linear functions of those latent parameters; the Gaussianprocess kernel functions define priors on the function space. Our generativemodel is trained with a justifiable likelihood function that allows us to treatheteroscedastic noise and missing data correctly, which is crucial for allastrophysical applications. It can predict simultaneously unobserved spectralregions, as well as the physical properties of quasars in held-out test data.We apply the model to rest-frame ultraviolet and optical quasar spectra forwhich precise black hole masses (based on reverberation mapping measurements)are available. Unlike reverberation-mapping studies, which require multi-epochdata, our model predicts black hole masses from single-epoch spectra, even withlimited spectral coverage. We demonstrate the capabilities of the model bypredicting black hole masses and unobserved spectral regions. We find that wepredict black hole masses at close to the best possible accuracy.|


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2212.09766-b31b1b.svg)](https://arxiv.org/abs/2212.09766) | **Comparing the Locations of Supernovae to CO (2-1) Emission in their Host Galaxies**  |
|| Ness Mayker Chen, et al. -- incl., <mark>Annie Hughes</mark>, <mark>Kathryn Kreckel</mark>, <mark>Frank Bigiel</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2022-12-19*|
|*Comments*| *Accepted to ApJ; 38 pages, 12 figures, 7 tables in two-column AASTEX63 format*|
|**Abstract**| We measure the molecular gas environment near recent ($< 100$ yr old)supernovae (SNe) using $\sim1''$ or $\leq 150$pc resolution CO (2-1) maps fromthe PHANGS-ALMA survey of nearby star-forming galaxies. This is arguably thefirst such study to approach the scales of individual massive molecular clouds($M_{\rm mol} \gtrsim 10^{5.3}$ M$_{\odot}$). Using the Open Supernova Catalog(OSC), we identify 63 SNe within the PHANGS-ALMA footprint. We detect CO (2-1)emission near $\sim60\%$ of the sample at 150pc resolution, compared to$\sim35\%$ of map pixels with CO (2-1) emission, and up to $\sim95\%$ of theSNe at 1kpc resolution compared to $\sim80\%$ of map pixels with CO (2-1)emission. We expect the $\sim60\%$ of SNe within the same 150pc beam as a GMCwill likely interact with these clouds in the future, consistent with theobservation of widespread SN-molecular gas interaction in the Milky Way, whilethe other $\sim40\%$ of SNe without strong CO (2-1) detections will deposittheir energy in the diffuse interstellar medium (ISM), perhaps helping drivelarge-scale turbulence or galactic outflows. Broken down by type, we detect CO(2-1) emission at the sites of $\sim85\%$ of our 9 stripped-envelope SNe(SESNe), $\sim40\%$ of our 34 Type II SNe, and $\sim35\%$ of our 13 Type IaSNe, indicating that SESNe are most closely associated with the brightest CO(2-1) emitting regions in our sample. Our results confirm that SN explosionsare not restricted to only the densest gas, and instead exert feedback across awide range of molecular gas densities.|

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2204.00342-b31b1b.svg)](https://arxiv.org/abs/2204.00342) | **Magnetism, rotation, and nonthermal emission in cool stars -- Average magnetic field measurements in 292 M dwarfs**  |
|| A. Reiners, et al. -- incl., <mark>M. Kürster</mark> |
|*Appeared on*| *2022-04-01*|
|*Comments*| *A&A accepted, abstract abbreviated, 20 pages, Table B.1 included in arXiv version, full MCMC posteriors and linefit plots are available at this http URL*|
|**Abstract**| Stellar dynamos generate magnetic fields that are of fundamental importanceto the variability and evolution of Sun-like and low-mass stars, and for thedevelopment of their planetary systems. We report measurements ofsurface-average magnetic fields in 292 M dwarfs from a comparison withradiative transfer calculations; for 260 of them, this is the first measurementof this kind. Our data were obtained from more than 15,000 high-resolutionspectra taken during the CARMENES project. They reveal a relation betweenaverage field strength, <B>, and Rossby number, $Ro$, resembling thewell-studied rotation-activity relation. Among the slowly rotating stars, wefind that magnetic flux, $\Phi_\textrm{B}$, is proportional to rotation period,$P$, and among the rapidly rotating stars that average surface fields do notgrow significantly beyond the level set by the available kinetic energy.Furthermore, we find close relations between nonthermal coronal X-ray emission,chromospheric H$\alpha$ and Ca H&K emission, and magnetic flux. Taken together,these relations demonstrate empirically that the rotation-activity relation canbe traced back to a dependence of the magnetic dynamo on rotation. We advocatethe picture that the magnetic dynamo generates magnetic flux on the stellarsurface proportional to rotation rate with a saturation limit set by theavailable kinetic energy, and we provide relations for average field strengthsand nonthermal emission that are independent of the choice of the convectiveturnover time. We also find that Ca H&K emission saturates at average fieldstrengths of $\langle B \rangle \approx 800$ G while H$\alpha$ and X-rayemission grow further with stronger fields in the more rapidly rotating stars.This is in conflict with the coronal stripping scenario predicting that in themost rapidly rotating stars coronal plasma would be cooled to chromospherictemperatures.|
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure P_M.eps</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2203.15822-b31b1b.svg)](https://arxiv.org/abs/2203.15822) | **The Fornax3D project: discovery of ancient massive merger events in the Fornax cluster galaxies NGC 1380 and NGC 1427**  |
|| Ling Zhu, et al. -- incl., <mark>Annalisa Pillepich</mark>, <mark>Francesca Pinna</mark> |
|*Appeared on*| *2022-03-29*|
|*Comments*| *A&A accepted, 19 pages, 16 figures*|
|**Abstract**| We report the discovery of ancient massive merger events in the early-typegalaxies NGC 1380 and NGC 1427, members of the Fornax galaxy cluster. Bothgalaxies have been observed by the MUSE IFU instrument on the VLT, as part ofthe Fornax3D project. By fitting recently-developed population-orbitalsuperposition models to the observed surface brightness as well as stellarkinematic, age, and metallicity maps, we obtain the stellar orbits, age andmetallicity distributions of each galaxy. We then decompose each galaxy intomultiple orbital-based components, including a dynamically hot inner stellarhalo component which is identified as the relic of past massive mergers. Bycomparing to analogues from cosmological galaxy simulations, chiefly TNG50, wefind that the formation of such a hot inner stellar halo requires the mergerwith a now-destroyed massive satellite galaxy of $3.7_{-1.5}^{+2.7} \times10^{10}$ Msun (about $1/5$ of its current stellar mass) in the case of NGC 1380and of $1.5_{-0.7}^{+1.6} \times10^{10}$ Msun (about $1/4$ of its currentstellar mass) in the case of NGC 1427. Moreover, we infer that the last massivemerger in NGC 1380 happened $\sim10$ Gyr ago based on the stellar agedistribution of the re-grown dynamically cold disk, whereas the merger in NGC1427 ended $t\lesssim 8$ Gyr ago based on the stellar populations in its hotinner stellar halo. The major merger event in NGC 1380 is the first one withboth merger mass and merger time quantitatively inferred in a galaxy beyond theLocal Volume. Moreover, it is the oldest and most massive merger uncovered innearby galaxies so far.|
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure figure/figure3</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2301.08310-b31b1b.svg)](https://arxiv.org/abs/2301.08310) | **Gaia-ESO Survey: massive stars in the Carina Nebula. A new census of OB stars**  |
|| S. R. Berlanas, et al. -- incl., <mark>M. Bergemann</mark> |
|*Appeared on*| *2023-01-19*|
|*Comments*| *Accepted for publication in A&A, 62 pages, 8 figures*|
|**Abstract**| The Gaia-ESO survey sample of massive OB stars in the Carina Nebula consistsof 234 stars. The addition of brighter sources from the Galactic O-StarSpectroscopic Survey and additional sources from the literature allows us tocreate the most complete census of massive OB stars done so far in the region.It contains a total of 316 stars, being 18 of them in the background and fourin the foreground. Of the 294 stellar systems in Car OB1, 74 are of O type, 214are of non-supergiant B type and 6 are of WR or non-O supergiant (II to Ia)spectral class. We identify 20 spectroscopic binary systems with an O-starprimary, of which 6 are reported for the first time, and another 18 with aB-star primary, of which 13 are new detections. The average observeddouble-lined binary fraction of O-type stars in the surveyed region is 0.35,which represents a lower limit. We find a good correlation between thespectroscopic n-qualifier and the projected rotational velocity of the stars.The fraction of candidate runaways among the stars with and without then-qualifier is 4.4% and 2.4%, respectively, although non resolved double-linedbinaries can be contaminating the fast rotators sample.|
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure location_new2.eps</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2211.01474-b31b1b.svg)](https://arxiv.org/abs/2211.01474) | **Peering into the Young Planetary System AB Pic. Atmosphere, Orbit, Obliquity & Second Planetary Candidate**  |
|| P. Palma-Bifani, et al. -- incl., <mark>C. Desgrange</mark>, <mark>M. Feldt</mark>, <mark>M. Meyer</mark>, <mark>M. Samland</mark>, <mark>Y. Zhang</mark> |
|*Appeared on*| *2022-11-02*|
|*Comments*| *17 pages, 13 Figures, 6 Tables*|
|**Abstract**| We aim to revisit the system AB Pic which has a known companion at theexoplanet/ brown-dwarf boundary. We based this study on a rich set ofobservations to investigate the companion's orbit and atmosphere. We composed aspectrum of AB Pic b merging archival VLT/SINFONI K-band data, with publishedspectra at J and H-band (SINFONI) and Lp-band (Magellan-AO), and photometricmeasurements (HST and Spitzer). We modeled the spectrum with ForMoSA, based ontwo atmospheric models: ExoREM and BT-SETTL13. We determined the orbitalproperties of b fitting the astrometric measurements from NaCo (2003 and 2004)and SPHERE (2015). The orbital solutions favor a semi-major axis of $\sim$190auviewed edge-on. With Exo-REM, we derive a T$_{eff}$ of 1700$\pm$50K and surfacegravity of 4.5$\pm$0.3dex, consistent with previous works, and we report forthe first time a C/O ratio of 0.58$\pm$0.08 ($\sim$solar). The posteriors aresensitive to the wavelength interval and the family of models used. Given the2.1hr rotation period and our vsin(i) of $\sim$73km/s, we estimate for thefirst time the true obliquity to be $\sim$45 or $\sim$135deg, indicating asignificant misalignment between the planet's spin and orbit orientations.Finally, a proper motion anomaly between the Hipparcos and Gaia eDR3 comparedto our SPHERE detection limits and adapted radial velocity limits indicate theexistence of a $\sim$6M$_{Jup}$ inner planet orbiting from 2 to 10au(40-200mas). The possible existence of an inner companion and the likelymiss-alignment of the spin axis orientation strongly favor a formation path bygravitational instability or core accretion within a disk closer insidefollowed by dynamical interactions. Confirmation and characterization of planetc and access to a broader wavelength coverage for planet b will be essential toprobe the uncertainties associated with the parameters.|
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2204.02017-b31b1b.svg)](https://arxiv.org/abs/2204.02017) | **Systematic KMTNet Planetary Anomaly Search. IV. Complete Sample of 2019 Prime-Field**  |
|| Weicheng Zang, et al. -- incl., <mark>Chung-Uk Lee</mark>, <mark>Andrew Gould</mark>, <mark>Xiangyu Zhang</mark>, <mark>Seung-Lee Kim</mark>, <mark>Dong-Joo Lee</mark>, <mark>Yongseok Lee</mark> |
|*Appeared on*| *2022-04-05*|
|*Comments*| *Accepted by MNRAS*|
|**Abstract**| We report the complete statistical planetary sample from the prime fields($\Gamma \geq 2~{\rm hr}^{-1}$) of the 2019 Korea Microlensing TelescopeNetwork (KMTNet) microlensing survey. We develop the optimized KMTNetAnomalyFinder algorithm and apply it to the 2019 KMTNet prime fields. We find atotal of 14 homogeneously selected planets and report the analysis of threeplanetary events, KMT-2019-BLG-(1042,1552,2974). The planet-host mass ratios,$q$, for the three planetary events are $6.34 \times 10^{-4}, 4.89 \times10^{-3}$ and $6.18 \times 10^{-4}$, respectively. A Bayesian analysis indicatesthe three planets are all cold giant planets beyond the snow line of their hoststars. The 13 planets are basically uniform in $\log q$ over the range $-5.0 <\log q < -1.5$. This result suggests that the planets below $q_{\rm break} =1.7 \times 10^{-4}$ proposed by the MOA-II survey may be more common thanpreviously believed. This work is an early component of a large project todetermine the KMTNet mass-ratio function, and the whole sample of 2016--2019KMTNet events should contain about 120 planets.|
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [10]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [11]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.09168.md
    + _build/html/tmp_2212.09168/./figures/scouse2x2.png
    + _build/html/tmp_2212.09168/./figures/scouse2x2.png
    + _build/html/tmp_2212.09168/./figures/MIR_images_v5.png
exported in  _build/html/2301.04442.md
    + _build/html/tmp_2301.04442/./pics/selection/Sin_Quad_LS_data_3_subplot.png
    + _build/html/tmp_2301.04442/./pics/selection/p1_quad_res_quad_term_data_LS_no_label.png
    + _build/html/tmp_2301.04442/./pics/selection/p1_quad_res_data_LS_no_label.png
    + _build/html/tmp_2301.04442/./pics/selection/LS_photometry_MEarth_individually.jpeg
    + _build/html/tmp_2301.04442/./pics/selection/LS_photometry_selection.jpeg
    + _build/html/tmp_2301.04442/./pics/selection/significant_indices_LS_1409.jpeg
exported in  _build/html/2209.02725.md
    + _build/html/tmp_2209.02725/./1to1_0_f_ps_s42_HST_1220_5000_2A_band1fixed_noSNRcut_beta10_new_2kernels_cross_N30_D1889_L2_Q16_SNR_overlap.png
    + _build/html/tmp_2209.02725/./spectra_ps_s42_HST

exported in  _build/html/2301.03689.md
    + _build/html/tmp_2301.03689/./sed_2_halpha_vs_co_jun_11_2022.png
    + _build/html/tmp_2301.03689/./7plot_ngc1365.png
    + _build/html/tmp_2301.03689/./sinan_2plot_dec_6_2022.png
exported in  _build/html/2211.04048.md
    + _build/html/tmp_2211.04048/./initial_pt_profiles.png
    + _build/html/tmp_2211.04048/./chem_profiles.png
exported in  _build/html/2211.00657.md
    + _build/html/tmp_2211.00657/./Figures/05_Appendix/shell_model_parameters.png
    + _build/html/tmp_2211.00657/./Figures/03_Analysis/Mrk1044_QSO_spec_full.png
    + _build/html/tmp_2211.00657/./Figures/03_Analysis/spectroastrometry.png
exported in  _build/html/2211.13426.md
    + _build/html/tmp_2211.13426/./CMDs_NGC7496_v4_r.png
    + _build/html/tmp_2211.13426/./galaxy_plus_region.png
    + _build/html/tmp_2211.13426/./ngc7496_rgb_co_h_alpha_v2.png
exported in  _build/html/2210.06504.md
    + _build/html/tmp_2210.06504/./laefig_ionized_denpost_linear_COSMOSNavg.png
    + _b

exported in  _build/html/2204.03253.md
    + _build/html/tmp_2204.03253/./FigHXMT_ME_lcfit4.png
    + _build/html/tmp_2204.03253/./sin_nois_lc.png
    + _build/html/tmp_2204.03253/./Figall_LRT.png
exported in  _build/html/2210.12412.md
    + _build/html/tmp_2210.12412/./n_wda.png
    + _build/html/tmp_2210.12412/./berio_fig.png
    + _build/html/tmp_2210.12412/./time_sequence.png
exported in  _build/html/2204.00793.md
    + _build/html/tmp_2204.00793/./cii_spectra.png
    + _build/html/tmp_2204.00793/./comp1.png
    + _build/html/tmp_2204.00793/./co65_spectra.png
    + _build/html/tmp_2204.00793/./comp2.png
    + _build/html/tmp_2204.00793/./co76_spectra.png
    + _build/html/tmp_2204.00793/./comp3.png
    + _build/html/tmp_2204.00793/./radial_profile.png
    + _build/html/tmp_2204.00793/./radial_ratio_co_cii.png
    + _build/html/tmp_2204.00793/./figure_sflaw.png
    + _build/html/tmp_2204.00793/./figure_deficit.png
exported in  _build/html/2209.08106.md
    + _build/html/tmp_2209.081

exported in  _build/html/2203.16734.md
    + _build/html/tmp_2203.16734/./f2.png
    + _build/html/tmp_2203.16734/./f3.png
    + _build/html/tmp_2203.16734/./f7.png
exported in  _build/html/2212.01397.md
    + _build/html/tmp_2212.01397/./figure23.png
    + _build/html/tmp_2212.01397/./figure10.jpg
    + _build/html/tmp_2212.01397/./figure17.jpg
exported in  _build/html/2209.15110.md
    + _build/html/tmp_2209.15110/./Fig4New.png
    + _build/html/tmp_2209.15110/./Fig2New.png
    + _build/html/tmp_2209.15110/./HAWCFeb2020.png
exported in  _build/html/2302.04507.md
    + _build/html/tmp_2302.04507/./stacked_biases.png
    + _build/html/tmp_2302.04507/./all_dcls_estimators.png
    + _build/html/tmp_2302.04507/./bias_contours_lcdm.png
exported in  _build/html/2211.12613.md
    + _build/html/tmp_2211.12613/./J1120_bin_spec_corr_sky_zoomin.png
    + _build/html/tmp_2211.12613/./xhi_other_constrain_cut_misc_scale.png
    + _build/html/tmp_2211.12613/./J1120_flux_distribution.png
exported in 

exported in  _build/html/2212.02627.md
    + _build/html/tmp_2212.02627/./w33/vf.png
    + _build/html/tmp_2212.02627/./w33/vg.png
    + _build/html/tmp_2212.02627/./w33/bts.png
exported in  _build/html/2211.13146.md
    + _build/html/tmp_2211.13146/./posterimage.png
    + _build/html/tmp_2211.13146/./2_targetvsTNGintarget_Exclusion.png
    + _build/html/tmp_2211.13146/./4_finalsample.png
exported in  _build/html/2204.01245.md
    + _build/html/tmp_2204.01245/figures/fig_spec_model.png
    + _build/html/tmp_2204.01245/figures/fig_velo_plot_first.png
    + _build/html/tmp_2204.01245/figures/fig_velo_plot_last.png
    + _build/html/tmp_2204.01245/figures/fig_halo_flux.png
exported in  _build/html/2211.16510.md
exported in  _build/html/2211.03641.md
    + _build/html/tmp_2211.03641/./fig/610.png
    + _build/html/tmp_2211.03641/././fig/CAMDs-dense.png
    + _build/html/tmp_2211.03641/./fig/lb.png
exported in  _build/html/2209.12906.md
    + _build/html/tmp_2209.12906/./figs/results/totglo